In [ ]:
import sys
sys.path.append('../')
sys.path.append('../tests/python-support/')

%matplotlib notebook

In [ ]:
import matplotlib
import experiments
import itertools
import util

from pprint import pprint

In [ ]:
# Set up details of experiments
end_time = [0.5]
sample_count = [ 5001 ]
h = [ 0.1000, 0.0750, 0.0625 ]

experiment_names = [ "homogeneous_dirichlet_unit_square",
                     "inhomogeneous_dirichlet_unit_square" ]
param_collection = itertools.product(end_time, sample_count, h)
params = [ { 'end_time': x[0], 'sample_count': x[1], 'mesh_resolution': x[2] }
          for x in param_collection]
experiment_input = [ { 'experiment': e, 'parameters': p }
                     for e, p in itertools.product(experiment_names, params) ]

In [ ]:
# Run experiments and collect results
results = experiments.run_experiments(experiment_input)

In [ ]:
pprint(results)

In [ ]:
grouped_results = itertools.groupby(results, key = lambda experiment: experiment['experiment'])
for experiment, result in grouped_results:
    result = list(result)
    num_dof = [ result['mesh']['num_vertices'] for result in result ]
    l2_error = [ result['error_summary']['l2'] for result in result ]
    h1_error = [ result['error_summary']['h1'] for result in result ]
    h1_semi_error = [ result['error_summary']['h1_semi'] for result in result ]
    l2_slope = util.estimate_slope(num_dof, l2_error)
    h1_slope = util.estimate_slope(num_dof, h1_error)
    
    print("Experiment name: {name}".format(name=experiment))
    print("Dof:      {}".format(num_dof))
    print("H1 semi:  {}".format(h1_semi_error))
    print("H1:       {}".format(h1_error))
    print("L2:       {}".format(l2_error))
    print("H1 slope: {}".format(h1_slope))
    print("L2 slope: {}".format(l2_slope))
    print("")

In [ ]:
util.estimate_slope(num_dof, l2_error)

In [ ]:
util.estimate_slope(num_dof, h1_error)